In [148]:
import argparse
import pandas as pd
import numpy as np
import math

def compute_gene_PSI(PSI_df,exon_gene_df,junction_gene_df,annot_gene_df,read_len):
    'Compute Percent Spliced In (PSI)'
    exons = list(exon_gene_df.index)
    min_reads = 10

    for exon in exons:

        exon_chrom = annot_gene_df.loc[exon]['probe.chr'].split('chr')[1]
        exon_start = annot_gene_df.loc[exon]['probe.start']
        exon_stop = annot_gene_df.loc[exon]['probe.stop']
        exon_len = math.fabs(exon_start - exon_stop) + 1
        exon_strand = annot_gene_df.loc[exon]['gene.strand']
        if exon_strand == '-':
            orig_start = exon_start
            exon_start = exon_stop
            exon_stop = orig_start

        B_intervals = []
        C_intervals = []
        for interval in junction_gene_df.index:
            chrom,start,stop = interval.split('_')
            if chrom != exon_chrom:
                print(chrom)
                print(exon_chrom)
                print('Chromosomes aren\'t matching, something is wrong!!')

            if int(start) < exon_start and int(stop) > exon_stop:#C
                C_intervals.append(interval)
            if int(start) < exon_start and (int(stop) >= exon_start and int(stop) <= exon_stop):#B1
                B_intervals.append(interval)
            if (int(start) >= exon_start and int(start) <= exon_stop) and int(stop) > exon_stop:#B2
                B_intervals.append(interval)

        A_reads = exon_gene_df.loc[exon]
        B_reads = junction_gene_df.loc[B_intervals]
        B_reads = B_reads.iloc[:,2:].sum()

        C_reads = junction_gene_df.loc[C_intervals]
        C_reads = C_reads.iloc[:,2:].sum()

        total_reads = A_reads + B_reads + C_reads
        if len(total_reads[total_reads < min_reads])/len(total_reads) > 0.2:
            continue
        else:
            A_B_norm = (A_reads + B_reads)/(read_len + exon_len-1)
            C_norm = C_reads/(read_len-1)

            mask = (A_B_norm == 0) & (C_norm == 0)
            PSI_norm = 100*((A_B_norm)/(A_B_norm + C_norm)) if (A_B_norm.sum() + C_norm.sum()) > 0.00 else 0
            PSI_norm = PSI_norm.where(~mask,0)
            PSI_df[exon] = PSI_norm

    return PSI_df

def max_coverage_samples(exon_df):
    'Given exon expression dataframe'
    'Take first two elements of sample name'
    'If there is duplicate sample'
    'Return list of max coverage non-dup samples'
    
    med_exon_df = pd.DataFrame(exon_df.median(axis=0))
    med_exon_df['samples'] = ["-".join(i.split('-')[:2]) for i in med_exon_df.index]
    med_exon_df['original_samples'] = med_exon_df.index
    highest_cov_samples = list(med_exon_df.groupby('samples').idxmax()[0]) 
    
    return highest_cov_samples

def filter_exons(PSI_df):
    'Given PSI dataframe'
    'Filter out exons with little variation:'
    'PSI = 100% or PSI = 0% in >90% patients'
    'Return filtered PSI'
    
    PSI_filtered_df = PSI_df.dropna(axis=1)
    exons = PSI_filtered_df.columns
    exons_to_remove = []
    num_samples = len(PSI_df.index)
    for exon in exons:
        exon_PSI = PSI_filtered_df[exon]
        num_zero = len(exon_PSI[exon_PSI == 0])
        num_hundred = len(exon_PSI[exon_PSI == 100])
        if num_zero/num_samples > 0.9 or num_hundred/num_samples > 0.9:
            exons_to_remove.append(exon)
    PSI_filtered_df = PSI_filtered_df.drop(exons_to_remove,axis=1).dropna()
    new_index = ["-".join(i.split('-')[:2]) for i in PSI_filtered_df.index]
    PSI_filtered_df.index = new_index

    return PSI_filtered_df

if __name__ == "__main__":
#     parser = argparse.ArgumentParser(description="Compute Percent Spliced In (PSI) for any exon")
#     parser.add_argument("--exon_expr", help="File path to exon expression dataset", type=str, required=True)
#     parser.add_argument("--junction_expr", help="File path to junction expression dataset", type=str, required=True)
#     parser.add_argument("--exprannot", help="Exon expression annotation file", type=str, required=True)
#     parser.add_argument("--chrom", help="Restrict analysis to this chromosome", type=str, required=False)
#     parser.add_argument("--min_samples", help="Require data for this many samples", type=int, default=0)
#     parser.add_argument("--read_len", help="Paired end length, default is GTEx read length: 75bp ", type=int, default=75)
#     parser.add_argument("--out", help="Write data files to this file", type=str, required=True)


#     args = parser.parse_args()
    EXONFILE = '/storage/dana/spliceSTR/expression/GTEx_Data_20150112_RNAseq_RNASeQCv1.1.8_exon_reads.lung.txt'#args.exon_expr
    JUNCTIONFILE = '/storage/dana/spliceSTR/expression/GTEx_Data_20150112_RNAseq_Flux1.6_junction_reads.lung.txt'#args.junction_expr
    ANNOTFILE = '/storage/dana/spliceSTR/expression/gencode.v19.annotations_exons.csv'#args.exprannot
    CHROM = '3'#args.chrom
    MINSAMPLES = 0#args.min_samples
    READLEN = 2*75 #args.read_len
    OUTFILE = 'chrom3_PSI.csv'#args.out

    exon_df = pd.read_table(EXONFILE, index_col=0)
    annot_df = pd.read_table(ANNOTFILE, index_col=0,sep=',')
    junction_df = pd.read_table(JUNCTIONFILE, index_col=0,
                                          sep='\t',
                                          skipinitialspace=True)
    if CHROM:
        if CHROM.startswith('chr'):
            CHROM = CHROM.split('chr')[1]
        annot_df = annot_df[annot_df['gene.chr'] == 'chr' + CHROM]
        junction_df =junction_df[junction_df['Gene_Symbol'].isin(annot_df['gene.id'])]
        exon_df = exon_df.loc[list(annot_df['probe.id'])]

    #removing duplicated samples by taking highest coverage (in exon expression) sample

    highest_cov_samples = max_coverage_samples(exon_df) 
    exon_df = exon_df[highest_cov_samples]
    highest_cov_samples[:0] = ['Gene_Symbol','Chr']
    junction_df = junction_df[highest_cov_samples]
    PSI_df = pd.DataFrame(index=exon_df.columns, columns=exon_df.index)
    genes = set(annot_df['gene.id'])
    print('Chromosome: {}'.format(CHROM))
    print('Exons total: {}'.format(len(exon_df.index)))
    
    for gene in genes:
        exon_gene_df = exon_df.loc[exon_df.index.str.startswith(gene)]
        junction_gene_df = junction_df[junction_df['Gene_Symbol'] == gene]
        annot_gene_df = annot_df[annot_df['gene.id'] == gene]
        annot_gene_df.index = annot_gene_df['probe.id']
        PSI_df = compute_gene_PSI(PSI_df,
                                  exon_gene_df,
                                  junction_gene_df,
                                  annot_gene_df,
                                  READLEN)
    PSI_filtered_df = filter_exons(PSI_df.dropna(axis=1))
    print('Exons with PSI: {}\n'.format(len(PSI_filtered_df.columns)))
    PSI_filtered_df.to_csv(OUTFILE)

/usr/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Chromosome: 3
Exons total: 14123
Exons with PSI: 2018



In [138]:
PSI_df.dropna(axis=1)

Id,ENSG00000172748.8_0,ENSG00000172748.8_6,ENSG00000147364.12_1,ENSG00000147364.12_2,ENSG00000147364.12_3,ENSG00000147364.12_4,ENSG00000147364.12_5,ENSG00000147364.12_6,ENSG00000147364.12_7,ENSG00000147364.12_8,...,ENSG00000170619.8_0,ENSG00000170619.8_1,ENSG00000196150.9_3,ENSG00000196150.9_5,ENSG00000196150.9_6,ENSG00000170631.10_0,ENSG00000182307.8_1,ENSG00000182307.8_2,ENSG00000182307.8_3,ENSG00000182307.8_4
GTEX-1117F-0226-SM-5GZZ7,100.0,100.0,100.0,100.0,0.000000,100.0,94.074377,100.0,100.0,100.0,...,21.450852,33.180750,48.961805,100.0,64.976751,100.0,100.0,98.883066,100.0,100.0
GTEX-111CU-0326-SM-5GZXO,100.0,100.0,100.0,100.0,0.000000,100.0,91.364234,100.0,100.0,100.0,...,9.370333,30.872626,43.976196,100.0,100.000000,100.0,100.0,98.025323,100.0,100.0
GTEX-111FC-1126-SM-5GZWU,100.0,100.0,100.0,100.0,0.000000,100.0,83.403679,100.0,100.0,100.0,...,13.571501,35.856711,75.075373,100.0,68.259162,100.0,100.0,99.161792,100.0,100.0
GTEX-111VG-0726-SM-5GIDC,100.0,100.0,100.0,100.0,0.000000,100.0,90.874490,100.0,100.0,100.0,...,4.748247,37.204629,54.677557,100.0,76.315603,100.0,100.0,100.000000,100.0,100.0
GTEX-111YS-0626-SM-5GZXV,100.0,100.0,100.0,100.0,0.000000,100.0,88.735926,100.0,100.0,100.0,...,8.410476,26.865789,68.253915,100.0,71.428361,100.0,100.0,99.192853,100.0,100.0
GTEX-1122O-0126-SM-5GICA,100.0,100.0,100.0,100.0,0.000000,100.0,90.193260,100.0,100.0,100.0,...,3.988436,32.289204,77.650858,100.0,57.845637,100.0,100.0,100.000000,100.0,100.0
GTEX-1128S-0726-SM-5N9D6,100.0,100.0,100.0,100.0,0.000000,100.0,78.267754,100.0,100.0,100.0,...,2.011339,34.109567,74.474958,100.0,66.945443,100.0,100.0,98.923085,100.0,100.0
GTEX-117YW-0526-SM-5H11C,100.0,100.0,100.0,100.0,0.000000,100.0,77.849658,100.0,100.0,100.0,...,13.426447,37.769328,71.843018,100.0,60.912623,100.0,100.0,100.000000,100.0,100.0
GTEX-117YX-1326-SM-5H125,100.0,100.0,100.0,100.0,0.000000,100.0,95.111186,100.0,100.0,100.0,...,4.327621,26.452150,69.941112,100.0,100.000000,100.0,100.0,98.651558,100.0,100.0
GTEX-11DXX-0626-SM-5Q5AG,100.0,100.0,100.0,100.0,0.000000,100.0,90.296337,100.0,100.0,100.0,...,10.823572,41.151641,74.444224,100.0,73.303058,100.0,100.0,97.343161,100.0,100.0


In [139]:
PSI_df

Id,ENSG00000253896.3_0,ENSG00000253896.3_1,ENSG00000176269.3_0,ENSG00000250210.3_0,ENSG00000250210.3_1,ENSG00000172748.8_0,ENSG00000172748.8_1,ENSG00000172748.8_2,ENSG00000172748.8_3,ENSG00000172748.8_4,...,ENSG00000170631.10_0,ENSG00000170631.10_1,ENSG00000170631.10_2,ENSG00000170631.10_3,ENSG00000170631.10_4,ENSG00000182307.8_0,ENSG00000182307.8_1,ENSG00000182307.8_2,ENSG00000182307.8_3,ENSG00000182307.8_4
GTEX-1117F-0226-SM-5GZZ7,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,98.883066,100.0,100.0
GTEX-111CU-0326-SM-5GZXO,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,98.025323,100.0,100.0
GTEX-111FC-1126-SM-5GZWU,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,99.161792,100.0,100.0
GTEX-111VG-0726-SM-5GIDC,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,100.000000,100.0,100.0
GTEX-111YS-0626-SM-5GZXV,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,99.192853,100.0,100.0
GTEX-1122O-0126-SM-5GICA,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,100.000000,100.0,100.0
GTEX-1128S-0726-SM-5N9D6,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,98.923085,100.0,100.0
GTEX-117YW-0526-SM-5H11C,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,100.000000,100.0,100.0
GTEX-117YX-1326-SM-5H125,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,98.651558,100.0,100.0
GTEX-11DXX-0626-SM-5Q5AG,NaN,NaN,NaN,NaN,NaN,100.0,NaN,NaN,NaN,NaN,...,100.0,NaN,NaN,NaN,NaN,NaN,100.0,97.343161,100.0,100.0
